# Movie Review Classification
* We are going to use 'Word2Vec' technique to convert the movie review(text data) into vector. And use this vector data as input layer in or Neural network
* We are also going to use TensorFlow 2.* version, Keras


In [1]:
import tensorflow as tf
tf.__version__

'2.9.1'

In [2]:
#Importing other tensorflow libra

import tensorflow_datasets as tfds # to use tensorflow prebuilt datasets
import tensorflow_hub as hub # Contains presaved models for transfer learning

Lets view the list of all the vailable datasets

In [3]:
tfds.list_builders()

['abstract_reasoning',
 'accentdb',
 'aeslc',
 'aflw2k3d',
 'ag_news_subset',
 'ai2_arc',
 'ai2_arc_with_ir',
 'amazon_us_reviews',
 'anli',
 'arc',
 'asset',
 'assin2',
 'bair_robot_pushing_small',
 'bccd',
 'beans',
 'bee_dataset',
 'big_patent',
 'bigearthnet',
 'billsum',
 'binarized_mnist',
 'binary_alpha_digits',
 'blimp',
 'booksum',
 'bool_q',
 'c4',
 'caltech101',
 'caltech_birds2010',
 'caltech_birds2011',
 'cardiotox',
 'cars196',
 'cassava',
 'cats_vs_dogs',
 'celeb_a',
 'celeb_a_hq',
 'cfq',
 'cherry_blossoms',
 'chexpert',
 'cifar10',
 'cifar100',
 'cifar10_1',
 'cifar10_corrupted',
 'citrus_leaves',
 'cityscapes',
 'civil_comments',
 'clevr',
 'clic',
 'clinc_oos',
 'cmaterdb',
 'cnn_dailymail',
 'coco',
 'coco_captions',
 'coil100',
 'colorectal_histology',
 'colorectal_histology_large',
 'common_voice',
 'coqa',
 'cos_e',
 'cosmos_qa',
 'covid19',
 'covid19sum',
 'crema_d',
 'cs_restaurants',
 'curated_breast_imaging_ddsm',
 'cycle_gan',
 'd4rl_adroit_door',
 'd4rl_adr

We are going to use 'imdb_review' dataset

In [4]:
tfds.load('imdb_reviews')

{'train': <PrefetchDataset element_spec={'label': TensorSpec(shape=(), dtype=tf.int64, name=None), 'text': TensorSpec(shape=(), dtype=tf.string, name=None)}>,
 'test': <PrefetchDataset element_spec={'label': TensorSpec(shape=(), dtype=tf.int64, name=None), 'text': TensorSpec(shape=(), dtype=tf.string, name=None)}>,
 'unsupervised': <PrefetchDataset element_spec={'label': TensorSpec(shape=(), dtype=tf.int64, name=None), 'text': TensorSpec(shape=(), dtype=tf.string, name=None)}>}

Lets split the data into training set, cross validation set and test set. Generally split is 60% for training, 20 for cross validation and 20 % for testing.

Cross validation data is used for fine tuning the model

In [5]:
train_data, validation_data, test_data = tfds.load(name="imdb_reviews",split=('train[:60%]', 'train[60%:]', 'test'), as_supervised=True)

In [6]:
type(train_data)

tensorflow.python.data.ops.dataset_ops.PrefetchDataset

In [7]:
# Since our result dataset are not of type list or dictionary we will get data as below
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
print('train_examples_batch= %s' % (train_examples_batch))
print('train_labels_batch= %s' % (train_labels_batch))

train_examples_batch= tf.Tensor(
[b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."
 b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film wa

Above result shows 10 movie review with 10 corresponding labels. 1 = positive review, 0 = negative review

* Now need to convert our movie review text using Word2Vec to vectors so that we can use it in our model
* We are going to use TensroFlow Hub to use pretrained model of Word2Vec 


In [8]:
pretrained_model = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(pretrained_model, input_shape=[], dtype=tf.string, trainable=True)
# hub_layer is or input layer which converts text inti vectors

Lets try and convert movie review text to vector using hub_layer

In [9]:
train_examples_batch[:1]

<tf.Tensor: shape=(1,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."],
      dtype=object)>

In [10]:
hub_layer(train_examples_batch[:1])

<tf.Tensor: shape=(1, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ]],
      dtype=float32)>

So the pretrained Word2Vec model converted the movie review into a vector of shape(1,20). 

## Creating a model

In [11]:
model = tf.keras.Sequential()

model.add(hub_layer) # input layer
model.add(tf.keras.layers.Dense(16, activation = 'relu')) # 16 neurons in hidden layer
model.add(tf.keras.layers.Dense(1, activation = 'sigmoid')) # Since single callss classification using sigmoid function. Use 'softmax' for multclass classification
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 20)                400020    
                                                                 
 dense (Dense)               (None, 16)                336       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


## Compile the model

In [12]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

## Train the model

In [13]:
## Shuffle the samples, total 20 iterations, pass validation data for fine tuning, verbose =1 for printing the output
model.fit(train_data.shuffle(10000).batch(512),  
          epochs=20, 
          validation_data = validation_data.batch(512),
          verbose =1)

Epoch 1/20
30/30 [==============================] - 2s 49ms/step - loss: 0.7528 - accuracy: 0.5311 - val_loss: 0.6470 - val_accuracy: 0.6305
Epoch 2/20
30/30 [==============================] - 1s 40ms/step - loss: 0.6127 - accuracy: 0.6663 - val_loss: 0.5930 - val_accuracy: 0.6869
Epoch 3/20
30/30 [==============================] - 1s 42ms/step - loss: 0.5656 - accuracy: 0.7128 - val_loss: 0.5578 - val_accuracy: 0.7220
Epoch 4/20
30/30 [==============================] - 1s 38ms/step - loss: 0.5289 - accuracy: 0.7465 - val_loss: 0.5287 - val_accuracy: 0.7428
Epoch 5/20
30/30 [==============================] - 1s 38ms/step - loss: 0.4943 - accuracy: 0.7709 - val_loss: 0.4984 - val_accuracy: 0.7660
Epoch 6/20
30/30 [==============================] - 1s 39ms/step - loss: 0.4608 - accuracy: 0.7951 - val_loss: 0.4710 - val_accuracy: 0.7853
Epoch 7/20
30/30 [==============================] - 1s 38ms/step - loss: 0.4285 - accuracy: 0.8160 - val_loss: 0.4459 - val_accuracy: 0.7997
Epoch 8/20
30

## Testing the model

In [14]:
model.predict([['This the worst movie i have even seen']])

1/1 [==============================] - 0s 85ms/step


array([[0.04100979]], dtype=float32)

Since result is near to zero means its negative review

In [15]:
model.predict([['This is best movie of all time']])

1/1 [==============================] - 0s 24ms/step


array([[0.9162369]], dtype=float32)

Since result is near to one its positive review

In [16]:
# We can also predict for multiple review's at a time
model.predict(["This is the worst movie I have ever seen",
              "An excellent movie that I enjoyed a lot",
              "how can one make such a horrible movie? there is no story, acting direction everything is very poor"])

1/1 [==============================] - 0s 67ms/step


array([[0.06790049],
       [0.9981456 ],
       [0.01407862]], dtype=float32)